# 1. Set Up

In [1]:
!spack find -p tfel

-- linux-ubuntu20.04-x86_64 / gcc@9.4.0 -------------------------
tfel@4.0.0  /mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx
tfel@4.0.0  /mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-jjcwdu6cbil5dzqzjhjekn3jdzo3e6gc
==> 2 installed packages


In [2]:
# %env LD_LIBRARY_PATH=/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib

In [3]:
!echo "$(spack find -p tfel | awk '/\/mofem_install\// {print $NF "/lib"}')"

/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-mvfpqw7u4c23su7hj7g4leuwmykrjmcx/lib
/mofem_install/spack/opt/spack/linux-ubuntu20.04-x86_64/gcc-9.4.0/tfel-4.0.0-jjcwdu6cbil5dzqzjhjekn3jdzo3e6gc/lib


In [4]:
import math
import os
import re
import sys
import time
import json
from pathlib import Path
import subprocess
import zipfile
import pydantic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv
import gmsh

sys.path.append('/mofem_install/jupyter/thomas/mfront_example_test/src')

import setup
import core
import custom_models as cm
import utils as ut
import plotting
import calculations as calc
    
import matplotlib
matplotlib.rc('figure', figsize=(7, 7))

# 2. Simulation Parameters

In [5]:
#in MPa
def initialize_parameters(soil_model: cm.PropertyTypeEnum = cm.PropertyTypeEnum.le,**kwargs) -> cm.AttrDict:
    params = cm.AttrDict()
    setattr(params, "soil_model", soil_model)
    for key,value in kwargs.items():
        setattr(params, key, value)
    params.tester = cm.TestAttr(
        preferred_model= params.soil_model,
        props = {
            cm.PropertyTypeEnum.le: cm.ElasticProperties(youngs_modulus=233, poisson_ratio=0.3),
            cm.PropertyTypeEnum.vM: cm.VonMisesProperties(youngs_modulus=233, poisson_ratio=0.3, HardeningSlope = 233/100, YieldStress = 0.3),
            cm.PropertyTypeEnum.dp: cm.DruckerPragerProperties(youngs_modulus=233, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15)),
            cm.PropertyTypeEnum.dpHYPER: cm.DPHYPERProps(youngs_modulus=233, poisson_ratio=0.3, phi=np.radians(15), c=2, v=np.radians(15), proximity = 20),
            cm.PropertyTypeEnum.mcc: cm.CamClayProperties(),
            }, 
        
    )
 
    
    params.nproc = 8 # number of processors/cores used
    params.order = 2 #order of approximation functions
    params.dim = 3

    
    
    params.wk_dir = Path(f"/mofem_install/jupyter/thomas/mfront_example_test")
    params.um_view = f"/mofem_install/jupyter/thomas/um_view"
    params.read_med_exe = "/mofem_install/jupyter/thomas/um_view/bin/read_med"
    params.h5m_to_vtk_converter = "/mofem_install/jupyter/thomas/um_view/bin/convert.py"
    params.partition_exe = "/mofem_install/jupyter/thomas/um_view/bin/mofem_part"
    params.exe = f"/mofem_install/jupyter/thomas/um_view/tutorials/adv-1/contact_3d"
    params.paraview_path = "/mofem_install/jupyter/thomas/ParaView-5.13.1-MPI-Linux-Python3.10-x86_64/bin/pvpython"
    params.ffmpeg_path = '/mofem_install/jupyter/thomas/ffmpeg-7.0.2-amd64-static/ffmpeg'
    params.preset_dir = '/mofem_install/jupyter/thomas/mfront_example_test/src/presets'
    params.options_file = "/mofem_install/jupyter/thomas/mfront_example_test/param_file.petsc"
    
    return params

In [6]:
def postprocess_extract(params):
    # Example fields and components to extract
    fields = ["STRAIN", "STRESS", "DISPLACEMENT"]
    components = {
        "STRAIN": 9,
        "STRESS": 9,
        "DISPLACEMENT": 3
    }

    # Initialize a dictionary to store DataFrames for each point ID
    point_dataframes = {}

    # Get the list of VTK files
    vtk_files = subprocess.run(
        f"ls -c1 {params.vtk_dir}/*.vtk | sort -V", 
        shell=True, text=True, capture_output=True
    )
    vtk_files_list = vtk_files.stdout.splitlines()
    print(vtk_files_list)

    # Iterate through the VTK files
    for i, vtk_file_path in enumerate(vtk_files_list):
        print(f"Processing VTK file: {vtk_file_path}")

        vtk_file = params.vtk_dir / f'out_mi_{i}.vtk'
        if not os.path.exists(vtk_file):
            print(f"File {vtk_file} does not exist.")
            continue

        # Read the VTK file
        mesh = pv.read(vtk_file)

        # Iterate through the points of interest
        for point in params.points_of_interest:
            point_coords = point.array()

            # Find all point IDs that match the coordinates
            all_point_ids = np.where(np.all(mesh.points == point_coords, axis=1))[0]
            if len(all_point_ids) == 0:
                print(f"No points found at coordinates {point_coords} in file {vtk_file}")
                continue

            # Initialize DataFrames for each point ID if not already done
            for pid in all_point_ids:
                if pid not in point_dataframes:
                    data = {f"{field}_{comp}": [] for field in fields for comp in range(components[field])}
                    data['time'] = []
                    data['x'] = []
                    data['y'] = []
                    data['z'] = []
                    point_dataframes[pid] = pd.DataFrame(data)

            # Append data for each point ID
            for pid in all_point_ids:
                data = {f"{field}_{comp}": [] for field in fields for comp in range(components[field])}
                data['time'] = i  # Add a time column
                data['x'] = point.x
                data['y'] = point.y
                data['z'] = point.z

                for field_name in fields:
                    point_data = mesh.point_data[field_name][pid]
                    for comp in range(components[field_name]):
                        data[f"{field_name}_{comp}"].append(point_data[comp])

                df = pd.DataFrame(data)
                point_dataframes[pid] = pd.concat([point_dataframes[pid], df], ignore_index=True)

    # Save the extracted data to a CSV file
    for pid, df in point_dataframes.items():
        output_path = cm.Point(x=df["x"].iloc[0], y=df["y"].iloc[0], z=df["z"].iloc[0]).point_dir(params) / f"point_data_{pid}.csv"
        point_dataframes[pid].to_csv(output_path, index=False)
        print(f"Saved data for point ID {pid} to {output_path}")


In [7]:
def postprocess_calculate(params):
    for point in params.points_of_interest:
        csv_files = [f for f in os.listdir(point.point_dir(params)) if f.startswith("point_data_") and f.endswith(".csv")]
        
        for csv_file in csv_files:
            pid = int(csv_file.split('_')[2].split('.')[0])
            df = pd.read_csv(point.point_dir(params) / csv_file)
            
            sig_xx = df['STRESS_0']
            sig_xy = df['STRESS_1']
            sig_xz = df['STRESS_2']
            sig_yy = df['STRESS_4']
            sig_yz = df['STRESS_5']
            sig_zz = df['STRESS_8']
            
            sig_1, sig_2, sig_3 = calc.calculate_principal_stresses(sig_xx, sig_yy, sig_zz, sig_xy, sig_yz, sig_xz)
            
            df['SIG_1'] = sig_1
            df['SIG_2'] = sig_2
            df['SIG_3'] = sig_3
            
            p = calc.calculate_p(sig_1, sig_2, sig_3)
            J_2 = calc.calculate_J2(sig_1, sig_2, sig_3)
            J  = np.sqrt(J_2)
            tau_oct = np.sqrt(2 * J_2)
            df["SIG_eq"] = np.sqrt(3 * J_2)
            
            e_xx = df['STRAIN_0']
            e_xy = df['STRAIN_1']
            e_xz = df['STRAIN_2']
            e_yy = df['STRAIN_4']
            e_yz = df['STRAIN_5']
            e_zz = df['STRAIN_8']
            
            e_v, e_d = calc.calculate_volumetric_and_deviatoric_strain(e_xx, e_yy, e_zz, e_xy, e_xz, e_yz)
            df['e_v'] = e_v
            df['e_d'] = e_d
            
            def calculate_stress_magnitude(row):
                stress_tensor = np.array([
                    [row['STRESS_0'], row['STRESS_1'], row['STRESS_2']],
                    [row['STRESS_1'], row['STRESS_4'], row['STRESS_5']],
                    [row['STRESS_2'], row['STRESS_5'], row['STRESS_8']]
                ])
                return np.linalg.norm(stress_tensor)
            
            def calculate_strain_magnitude(row):
                strain_tensor = np.array([
                    [row['STRAIN_0'], row['STRAIN_1'], row['STRAIN_2']],
                    [row['STRAIN_1'], row['STRAIN_4'], row['STRAIN_5']],
                    [row['STRAIN_2'], row['STRAIN_5'], row['STRAIN_8']]
                ])
                return np.linalg.norm(strain_tensor)
            
            def calculate_displacement_magnitude(row):
                return np.sqrt(
                    row['DISPLACEMENT_0']**2 + 
                    row['DISPLACEMENT_1']**2 + 
                    row['DISPLACEMENT_2']**2
                )
            
            df['STRESS_magnitude'] = df.apply(calculate_stress_magnitude, axis=1)
            df['STRAIN_magnitude'] = df.apply(calculate_strain_magnitude, axis=1)
            df['DISPLACEMENT_magnitude'] = df.apply(calculate_displacement_magnitude, axis=1)
            
            df.to_csv(point.point_dir(params) /  f"point_data_{pid}.csv", index=False)

In [8]:
def postprocess_plot_new(params):
    data_force=pd.read_csv(params.FIX_X_1_force_log_file,sep='\s+',header=None)
    load = - data_force[4].values * 2 * (10 ** 6) / 1000
    
    for point in params.points_of_interest:
        csv_files = [f for f in os.listdir(point.point_dir(params)) if f.startswith("point_data_") and f.endswith(".csv")]
        for csv_file in csv_files:
            pid = int(csv_file.split('_')[2].split('.')[0])
            df = pd.read_csv(point.point_dir(params) / csv_file)

            # df_ground_level_passive = pd.read_csv("/mofem_install/jupyter/thomas/mfront_example_test/simulations/pile_day_104_sim_1_20241129_010700_vM/-1.0_0.0_0.0/point_data_71015.csv")
            disp_z = - np.array(df['DISPLACEMENT_2']) * 1000
            
            # Create a DataFrame from the arrays
            df_final = pd.DataFrame({
                'disp_z': disp_z,
                'load': load
            })
            # Save the DataFrame to a CSV file
            df_final.to_csv(f"{point.point_dir(params)}/disp_x_vs_pile_tip_lateral_load_{pid}.csv", index=False)
            
            plotting.plot_x_ys(disp_z, [load], labels=["FEA"], x_label='Veritcal displacement$\mu_z$ [mm]', y_label='Load $H$ [kN]', title='Load $H$ vs $\mu_z$', save_as = f"{point.point_dir(params)}/412_H_uz_{pid}.png", show=True)

In [ ]:
params = initialize_parameters(
    case_name = "test_3D",
    base = "hex",
    soil_model=cm.PropertyTypeEnum.dp,
    
    final_time = 1, # [s]
    time_step = 1 / 100, # [s]
    options_file = "/mofem_install/jupyter/thomas/mfront_example_test/param_file.petsc",
    
    save_gauss = 0,
    
    FEA_completed=True,
    days_since_epoch=105,
    sim_otd=29,
    
    # the prescribed force/dosplacement is always applied to the max_z surface
    # params.prescribed_force = cm.ForceBoundaryCondition(f_x=0,f_y=0,f_z=60)
    prescribed_disp = cm.EdgeBoundaryCondition(disp_uz=1),
    
    time_history = cm.TimeHistory(history = {
        0: 0, 
        # 0.2: -0.15, 
        1: -0.5,
        }
    ),
    
    mesh_size = int(10),
    
    points_of_interest = [
        cm.Point(x=5,y=5,z=10),
        cm.Point(x=-5,y=-5,z=0),
    ],
    line_of_interest = cm.Line(pt1=cm.Point(x=5,y=5,z=10), pt2=cm.Point(x=5,y=5,z=0),),
) # 2. Simulation Parameters
params = setup.setup(params) # 3. Log paths and meta
if params.FEA_completed:
    subprocess.run(f"grep 'Total force:' {params.log_file} > {params.total_force_log_file}", shell=True)
    subprocess.run(
        f"grep -A 2 'FIX_X_1' {params.log_file} | awk '/Force/' > {params.FIX_X_1_force_log_file}",
        shell=True
    )
    # subprocess.run(f"grep 'Force:' {params.log_file} > {params.force_log_file}", shell=True)
    subprocess.run(f"grep 'nb global dofs' {params.log_file} > {params.DOFs_log_file}", shell=True)
    
    # core.export_to_vtk(params)
    postprocess_extract(params)
    postprocess_calculate(params)
    postprocess_plot_new(params)
    # postprocessing(params) # 7. Extract data from .vtk files 
    # plot_all_and_auxiliary_saves(params) # 8. Plotting
    pass
else:
    core.generate_mesh(params) # 4. Generate the mesh
    core.mofem_compute(params) # 5. Running the analysis and export to .vtk file format
    core.export_to_vtk(params)
    postprocess_extract(params)
    postprocess_calculate(params)
    postprocess_plot_new(params)

╭────────────────────────────────╮
│  CONVERTING FROM .h5m TO .vtk  │
╰────────────────────────────────╯
 |█████████████████---------------------------------| 34.1% (28 of 82)

 |██████████████████████████████████████████████████| 100.0% (82 of 82)

Conversion to VTK successful.
Moved out_mi_79.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_78.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_9.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_77.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_81.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_76.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/vtks
Moved out_mi_80.vtk to /mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_2

FileNotFoundError: [Errno 2] No such file or directory: '/mofem_install/jupyter/thomas/mfront_example_test/simulations/test_3D_day_105_sim_29_20241130_164216_dp/result_test_3D_day_105_sim_29_dp_FIX_X_1_force.log'

In [ ]:
for point in params.points_of_interest:
    csv_files = [f for f in os.listdir(point.point_dir(params)) if f.startswith("point_data_") and f.endswith(".csv")]
    for csv_file in csv_files:
        pid = int(csv_file.split('_')[2].split('.')[0])
        df = pd.read_csv(point.point_dir(params) / csv_file)
        sig_1 = np.array(df['SIG_1'])
        sig_2 = np.array(df['SIG_2'])
        sig_3 = np.array(df['SIG_3'])

        x_1 = sig_1
        y_1 = sig_2
        z_1 = sig_3

        # Convert lists to JavaScript arrays
        x_1_js = ', '.join(map(str, x_1))
        y_1_js = ', '.join(map(str, y_1))
        z_1_js = ', '.join(map(str, z_1))
        interacative_html_js = f"""
<!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Desmos Calculator Debug</title>
            <script src="https://www.desmos.com/api/v1.11/calculator.js?apiKey=dcb31709b452b1cf9dc26972add0fda6"></script>
        </head>
        <body>
        <div id="calculator" style="width: 1200px; height: 600px;"></div>
        <script>
            var elt = document.getElementById('calculator');
            var calculator = Desmos.Calculator3D(elt);"""
        if params.soil_model == cm.PropertyTypeEnum.le:
            pass
        elif params.soil_model == cm.PropertyTypeEnum.vM:
            sig_y = 10
            H = 0
            interacative_html_js += f"""
                calculator.setExpression({{id:'exp1', latex: 'I = x + y + z'}});
                calculator.setExpression({{id:'exp2', latex: 'p = I / 3'}});
                calculator.setExpression({{id:'exp3', latex: 'J_2= \\\\frac{{1}}{{6}} \\\\cdot ((x-y)^2 + (y-z)^2 + (z-x)^2) '}});
                calculator.setExpression({{id:'exp4', latex: 'q = \\\\sqrt{{3 \\\\cdot J_2}}'}});
                calculator.setExpression({{id:'exp5', latex: 's_{{0}}={sig_y}'}});
                calculator.setExpression({{id:'exp6', latex: 'H = {H}'}});

                calculator.setExpression({{id:'exp7', 
                    latex: '0 = q - s_{{0}}',
                    color: Desmos.Colors.RED,
                }});"""
        elif params.soil_model == cm.PropertyTypeEnum.dpHYPER or params.soil_model == cm.PropertyTypeEnum.dp:
            c = 10
            # a = 1e-16
            phi = 15 * math.pi / 180
            
            # Combine HTML and JavaScript to create interactive content within the notebook
            interacative_html_js += f"""
                calculator.setExpression({{id:'exp1', latex: 'I = x + y + z'}});
                calculator.setExpression({{id:'exp2', latex: 'p = I / 3'}});
                calculator.setExpression({{id:'exp3', latex: 'J_2= \\\\frac{{1}}{{6}} \\\\cdot ((x-y)^2 + (y-z)^2 + (z-x)^2) '}});
                calculator.setExpression({{id:'exp4', latex: 'q = \\\\sqrt{{3 \\\\cdot J_2}}'}});
                calculator.setExpression({{id:'exp5', latex: 'p_{{hi}} = {phi}'}});
                calculator.setExpression({{id:'exp6', latex: 'M_{{JP}} = \\\\frac{{2\\\\sqrt{{3}}\\\\sin p_{{hi}}}}{{3-\\\\sin p_{{hi}}}}'}});
                calculator.setExpression({{id:'exp7', latex: 'a = 10^{{-12}}'}});
                calculator.setExpression({{id:'exp8', latex: 'c = {c}'}});

                calculator.setExpression({{id:'exp9', 
                    latex: '0 = + M_{{JP}} p + \\\\sqrt{{a^{{2}} M_{{JP}}^{{2}} + \\\\frac{{q}}{{\\\\sqrt{{3}}}}^{{2}}}} - M_{{JP}} \\\\cdot \\\\frac{{c}}{{\\\\tan p_{{hi}}}}',
                    color: Desmos.Colors.RED,
                }});"""
        interacative_html_js += f"""
                calculator.setExpression({{
                    type: 'table',
                    columns: [
                        {{
                            latex: 'x_1',
                            values: [{x_1_js}]
                        }},
                        {{
                            latex: 'y_1',
                            values: [{y_1_js}],
                        }},
                        {{
                            latex: 'z_1',
                            values: [{z_1_js}],
                        }},
                    ]
                }});

                calculator.setExpression({{id:'exp20', 
                    latex: '(x_{{1}},y_{{1}},z_{{1}})',
                    color: Desmos.Colors.BLUE,
                }});
                
                
                function downloadScreenshot() {{
                    var screenshot = calculator.screenshot();
                    var link = document.createElement('a');
                    link.href = screenshot;
                    link.download = 'screenshot.png';
                    document.body.appendChild(link);
                    link.click();
                    document.body.removeChild(link);
                }}
                
            </script>
            <h2>Interactive Content</h2>
            <button onclick="downloadScreenshot()">Click me to download screenshot!</button>
            </body>
            """
        Html_file= open(f"{point.point_dir(params)}/Desmos3D_{pid}.html","w")
        Html_file.write(interacative_html_js)
        Html_file.close()
        